 Todo:
 1. check the order of the preprocess. what to do first === DONE ===
 2. create pipeline 
 3. hyper param search
 4. after findind hyper-params, add fit on all data (include val) 
 5. remove noisy samples: one option - remove rows with num of nan > trash
 6. feature selection (annova, pca) 
 7. check for cat boost 
 8. 

In [1]:
############# READ DATA
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from copy import copy

In [2]:
TRAIN_PATH = 'input/saftey_efficay_myopiaTrain.csv'
TEST_PATH = 'input/saftey_efficay_myopiaTest.csv'

train = pd.read_csv(TRAIN_PATH, low_memory=False)
test = pd.read_csv(TEST_PATH, low_memory=False)

train = train.dropna(axis=0, how='all') # remove missing rows
y_train = train.Class
columns_to_drop = ['Class', 'Pre_L_Pupil_Day', 'T_L_Actual_AblDepth']
train = train.drop(columns=columns_to_drop)
columns_to_drop = ['Pre_L_Pupil_Day', 'T_L_Actual_AblDepth']
test = test.drop(columns=columns_to_drop)

print("train.shape without missing rows =", train.shape)
print("class count:\n", y_train.value_counts())

train.shape without missing rows = (30451, 49)
class count:
 0.0    29949
1.0      502
Name: Class, dtype: int64


In [3]:
#### preprocess
import warnings
warnings.filterwarnings('ignore')

################### concat test train
n_train = train.shape[0]
concat = pd.concat(objs=[train, test], axis=0, sort=False)
concat_dummies = pd.get_dummies(concat)
train_dummies = copy(concat_dummies[:n_train])
test_dummies = copy(concat_dummies[n_train:])
assert np.array_equal(train_dummies.columns, test_dummies.columns), "columns not the same"

In [4]:
# X = (dummies-dummies.min())/(dummies.max()-dummies.min())
scaler = StandardScaler()
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

X = scaler.fit_transform(train_dummies)
# X = mean_imputer.fit_transform(X)
test = scaler.transform(test_dummies)
# test = mean_imputer.transform(test)
assert np.array_equal(train_dummies.columns, test_dummies.columns), "columns not the same"

In [5]:
x_train, x_val, y_train, y_val = train_test_split(X, y_train, test_size = 0.1, random_state=12)

sm = SMOTE(random_state=12, ratio = 1.0)
x_train_res, y_train_res = sm.fit_sample(x_train, y_train)  

In [6]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', n_jobs=-1, max_depth=8)
rf.fit(x_train_res, y_train_res)
rf_pred = rf.predict_proba(x_val)[:, 1]
print("random forest auc: ", roc_auc_score(y_val, rf_pred))

random forest auc:  0.5283447398703894


In [10]:
xg = XGBClassifier(n_jobs=-1, n_estimators=300, max_depth=8)
xg.fit(x_train_res, y_train_res)
xg_pred = xg.predict_proba(x_val)[:, 1]
print("xgboost auc: ", roc_auc_score(y_val, xg_pred))

xgboost auc:  0.5279359215068742


In [8]:
xg_test = xg.predict_proba(test)[:,1]
# rf_test = rf.predict_proba(test)[:,1]

In [9]:
ids = np.arange(1, test.shape[0]+1)
xg_df = pd.DataFrame({"Id" : ids, "Class" : xg_test})
rf_df = pd.DataFrame({"Id" : ids, "Class" : rf_test})
xg_df.to_csv("out/xg_out.csv", index=False)
rf_df.to_csv("out/rf_out.csv", index=False)